## Langchain + OpenAI LLM tracing using Maxim + Langchain

In [1]:

from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

True

In [2]:
import os

LOG_REPOSITORY_ID = os.getenv("MAXIM_LOG_REPO_ID")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

Here we will initialize MaximLangchainTracer

In [3]:
from maxim import Maxim, Config, LoggerConfig

logger = Maxim(Config()).logger(LoggerConfig(id=LOG_REPOSITORY_ID))

[MaximSDK] Using global logging level
[MaximSDK] Initializing Maxim AI(v3.6.3)


/var/folders/p0/3mgzlyg541g1j9s2d1wskc040000gn/T/ipykernel_13149/944686778.py:3: DeprecationWarning: This class will be removed in a future version. Use {} which is TypedDict.
  logger = Maxim(Config()).logger(LoggerConfig(id=LOG_REPOSITORY_ID))
/var/folders/p0/3mgzlyg541g1j9s2d1wskc040000gn/T/ipykernel_13149/944686778.py:3: DeprecationWarning: This class will be removed in a future version. Use LoggerConfigDict instead.
  logger = Maxim(Config()).logger(LoggerConfig(id=LOG_REPOSITORY_ID))


Setting up langchain call

In [4]:
import json
import uuid
from langchain_openai import ChatOpenAI
from maxim.logger.langchain import MaximLangchainTracer
from langchain_core.tools import tool

langchain_tracer = MaximLangchainTracer(logger)

MODEL_NAME = "gpt-4o"
llm = ChatOpenAI(model=MODEL_NAME, api_key=OPENAI_API_KEY)


# Define a simple tool
@tool
def search_tool(query: str) -> str:
     """
     Search the web for the latest news.

    Args:
        query: first int        
    """
     return f"Search results for: {query}"


user_input = "latest football news"
system_message = "You are a helpful assistant."

llm_with_tools = llm.bind_tools([search_tool])
trace_id = str(uuid.uuid4())
trace = logger.trace({"id": trace_id, "name": "langchain_trace"})
trace.end()
# Make the API call to Claude using LangChain
messages = [("system", system_message), ("human", user_input)]
response = llm_with_tools.invoke(messages,config={"callbacks":[langchain_tracer],"metadata":{"maxim":{"trace_id":trace.id}}})
print(response)
response_text = response.content
search_toll_call = trace.tool_call({"id": str(uuid.uuid4()),"name": "search_tool","args":json.dumps({"query":user_input})})
# execute the tool
final_tool_call_response = "this is tool result"
search_toll_call.result(final_tool_call_response)
print(response_text)
trace.set_output(final_tool_call_response)


# service 2
trace = logger.trace({"id": trace_id, "name": "langchain_trace"})
span_id=str(uuid.uuid4())
span = trace.span({"id": span_id,"name": "langchain_span"})

response = llm_with_tools.invoke(messages,config={"callbacks":[langchain_tracer],"metadata":{"maxim":{"span_id":span_id}}})
span.event(str(uuid.uuid4()),"langchain_event")
span.end()


content='' additional_kwargs={'tool_calls': [{'id': 'call_fWQN1t24FOCaLoSKHrtDbKeM', 'function': {'arguments': '{"query":"latest football news"}', 'name': 'search_tool'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 61, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_90122d973c', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-091811d6-2b78-44db-9b22-eaa694999414-0' tool_calls=[{'name': 'search_tool', 'args': {'query': 'latest football news'}, 'id': 'call_fWQN1t24FOCaLoSKHrtDbKeM', 'type': 'tool_call'}] usage_metadata={'input_tokens': 61, 'output_tokens': 16, 'total_tokens': 77}



In [ ]:
langchain_tracer = MaximLangchainTracer(logger)

MODEL_NAME = "gpt-4o"
llm = ChatOpenAI(model=MODEL_NAME, api_key=OPENAI_API_KEY, streaming=True)

user_input = "Describe big bang theory"
system_message = "You are a helpful assistant."
# Make the API call using LangChain with streaming
messages = [("system", system_message), ("human", user_input)]

# Stream the response
response_text = ""
for chunk in llm.stream(messages, config={"callbacks":[langchain_tracer]}):
    pass

print("\n\nFull response:", response_text)